# 8Pay valuation model

### Current Token Setup
8pay collects a fee of 1 \% of all payments made through their system. Half of those fees will go to 8pay and half will be redistributed among all stakers of the 8pay token on e.g. a weekly basis. As such, the 8pay token currently functions as a security token and its value depends entirely on the future cashflow provided by fees via staking, since its additional function of working as a means of exchange is satisfied in a superior fashion by stable coins such as TUSD or USDC.

### Prerequisits
To calculate the value of the entire token supply, we have to figure out the transaction volume (number of transactions times average transaction value) which serves as the basis from which fees are taken, along with a discount rate. The discount rate can include a lot of other factors, such as real interest rate (i.e. nominal interest rates minus inflation rate) or risk - and perhaps it should even take the yield curve into consideration.

### Modelling
The implemented model sums the discounted cashflows over the given time horizon and assumes that after that time horizon has been reached, cashflows remaining constant at the amount of the time horizon crossing. Compounding is done on a "per period" basis, with periods being either yearly, quarterly, monthly, or weekly. Rates are adjusted to always yield the specified discount rate on an annual basis. To model the transaction count, I chose a parameterized S-shaped curve with initial exponential growth that eventually saturates.

### Applicability
The main issues with this model are of course the uncertainty involved in the entire venture itself, the projection of the number of 8pay transactions and their average value, along with the interest rates that come to bear.

### Next Steps
This model is mainly intended as a toy model aimed at illustrating the approach that ought to be taken to calculate the fair value of the 8pay token at any point in time, given known data. To get to a better evaluation for investors, one could use this model to generate different scenarios (e.g. small, medium, and large scale adoption), assign probabilities to these scenarios and then do an expected value calculation based on those numbers.

In [1]:
import scipy.integrate as integrate
import numpy
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

We use a logistic function (see https://en.wikipedia.org/wiki/Logistic_function ) for our S-shaped curve, though any curve shape will work and one should look at references classes of other projects to estimate what a reasonable adoption curve might look like.

In [2]:
logistic = lambda t, mid, slope, limit: limit/(1+numpy.exp(-slope*(t-mid)))

Integrating the given span of time to find e.g. transaction volume yields more robust results than just doing e.g. linear interpolation.

In [3]:
average = lambda func, interval: integrate.quad(func, interval[0], interval[1])[0]

Back of the envelope calculation on the maximum number of available token transactions on the Ethereum network given its current technological state and the assumption of about 100k gas for a single token transfer.

In [4]:
blockGasLimit = 8000000
gasPerTransaction = 100000
blocksPerMinute = 4
annualMaxTransactions = numpy.floor(blockGasLimit/gasPerTransaction)*blocksPerMinute*60*24*365
printmd(f'**{annualMaxTransactions/10**6}M**')

**168.192M**

Primary function to calculate, sum, and plot the net present value of all future cashflows.

#### Parameters
* timeHorizon: $h$ number of years to be calculated - transaction volume past the time horizon is assumed to remain constant in perpetuity
* periodsPerYear: $p$ number of the staking periods per year - e.g. 1 $\leftrightarrow$ annually, 52 $\leftrightarrow$ weekly.
* discountRate: $d_a$ annual discount rate. adjusted by the number of periods per year $p$ to get the effective, per period discount rate $d_p$.
* stakerFee: $f$ what share of the fees belongs to 8pay token stakers (has a directly proportional effect on the outcome)
* mid: $\tau_m$ middle of the transaction rate S-curve $\tau$. The time when the exponential growth regime of the curve ends and the saturation regime starts.
* slope: $\tau_s$ determines the speed of growth and saturation of the transaction count S-curve $\tau$.
* limit: $\tau_l$ determine the upper, annualized transaction limit of the transaction count S-curve $\tau$.
* avgTransactionValue: $\nu$ average value of an 8pay transaction in USD (currently assumed to be constant and thus also impacts the overall outcome proportionally)

#### Model

Goal: We are looking for our total discounted cashflow $C$.

We calculate the per period discount rate $d_p$ via:
$$ d_p = \sqrt[p]{d_a} $$
(For example, a 10 % annual discount rate implies $d_a = 1.1$ and $d_p \approx 1.0241$)

We define our logistic transaction rate function $\tau(t)$ which yields the annualized rate of transactions at time $t$ via:
$$ \tau(t) := \frac{\tau_l}{1+e^{-\tau_s (t - \tau_m)}} $$

For simplicity's sake, we assume a constant transaction value function $\nu(t)$ (though as for the transaction rate function, an arbitrary curve could be used):
$$ \nu(t) := \nu $$

Now, we split our $h$ years until the time horizon into $p$ periods each, for a total of $h p$ periods.

For every period we calculate the transaction volume $v_i, i \in \{1, 2, \ldots, h p\}$:
$$ v_i = \int_{\frac{i}{p}}^{\frac{i+1}{p}} \tau(t) \nu(t) dt $$

The cashflow $\hat{c}_i$ for all stakers for period $i$ is then simply the staker fee $f$ time the transaction volume $v_i$:
$$ \hat{c}_i = f v_i $$

And finally, to arrive at the discounted cashflow $c_i$, we discount $\hat{c}_i$ $i$ times using our per period discount rate $d_p$:
$$ c_i = \frac{\hat{c}_i}{{d_p}^i} $$

Finally, to get our total discounted cashflow before the time horizon $C_{\leq h}$:
$$ C_{\leq h} = \sum_{i=1}^{h p} c_i $$

To calculate the total discounted cashflow after the time horizon $C_{>h}$, we assume that all future cashflows $\hat{c}_j, j \in \{hp+1, hp+2, \ldots\}$ are equal to $\hat{c}_{hp}$:
$$ \forall j \in \{hp+1, hp+2, \ldots\}: \hat{c}_j = \hat{c}_{hp}$$

Thus, all discounted post-horizon cashflows $c_j$ form a geometric series:
$$ c_j = \frac{\hat{c}_j}{{d_p}^j} = \frac{\hat{c}_{hp}}{{d_p}^j} \quad j \in \{hp+1, hp+2, \ldots \} $$
And hence:
$$ C_{>h} = \sum_{j=hp+1}^\infty c_j = \sum_{j=hp+1}^\infty \frac{\hat{c}_{hp}}{{d_p}^j} $$
Shifting the index by $hp+1$, we get:
$$ C_{>h} = \sum_{k=0}^\infty \frac{\hat{c}_{hp}}{{d_p}^{hp+1+k}} $$
Replacing $\frac{\hat{c}_{hp}}{{d_p}^{hp+1+k}}$ with $\frac{c_{hp}}{{d_p}^{1+k}}$ yields:
$$ C_{>h} = \sum_{k=0}^\infty \frac{c_{hp}}{{d_p}^{1+k}} $$
Finally, pulling out $\frac{c_{hp}}{d_p}$ and applying the formula for geometric sums $\sum_{i=0}^\infty q = \frac{1}{1-q}$ completes the simplification:
$$ C_{>h} = \frac{c_{hp}}{d_p} \sum_{k=0}^\infty \frac{1}{{d_p}^k} = \frac{c_{hp}}{d_p} \frac{1}{1-\frac{1}{d_p}} = \frac{c_{hp}}{d_p-1}$$

Total discounted cashflow $C$ is then the sum of both cashflows $C_{\leq h}$ and $C_{>h}$:
$$C = C_{\leq h} + C_{>h}$$

In [5]:
def valuation(timeHorizon,\
              periodsPerYear,\
              discountRate,\
              stakerFee,\
              mid,slope,limit,\
              avgTransactionValue,\
             ):
    discountRate = numpy.power(1+discountRate/100, 1/periodsPerYear)
    stakerFee = stakerFee/100
    transactionCount = lambda t: logistic(t, mid, slope, limit)
    transactionValue = lambda t: avgTransactionValue #for now constant
    periodTransactionCount = numpy.zeros(periodsPerYear*timeHorizon) #only for plotting
    periodTransactionVolume = numpy.zeros(periodsPerYear*timeHorizon)
    periodCashflow = numpy.zeros(periodsPerYear*timeHorizon)
    periodDiscounted = numpy.zeros(periodsPerYear*timeHorizon)
    timeAxis = numpy.zeros(periodsPerYear*timeHorizon)
    for i in range(periodsPerYear*timeHorizon):
        timeAxis[i] = i/periodsPerYear
        periodStartEnd = [i/periodsPerYear, (i+1)/periodsPerYear]
        periodTransactionCount[i] = average(transactionCount, periodStartEnd)
        periodTransactionVolume[i] = average(lambda t: transactionCount(t)*transactionValue(t), periodStartEnd)
        periodCashflow[i] = periodTransactionVolume[i]*stakerFee
        periodDiscounted[i] = periodCashflow[i]/numpy.power(discountRate,i+1)
        #print(f'time: {timeAxis[i]}, count: {periodTransactionCount[i]}, cashflow: {periodCashflow[i]}, discounted: {periodDiscounted[i]}')
    summedDiscounted = int(numpy.round(numpy.sum(periodDiscounted)/1000))
    discountedHorizon = int(numpy.round(periodDiscounted[-1]/((discountRate-1)*1000)))
    plt.plot(timeAxis, periodTransactionCount, )
    plt.xlabel('Years')
    plt.ylabel('Annualized Rate')
    plt.title('Transaction Rate')
    plt.grid(True)
    plt.show()
    plt.plot(timeAxis, periodCashflow, 'ob', timeAxis, periodDiscounted, '+r')
    plt.xlabel('Years')
    plt.ylabel('USD')
    plt.title('Cashflows (real and discounted)')
    plt.grid(True)
    plt.show()
    print(f'sum of pre-horizon discounted cashflows (C_<=h): $ {summedDiscounted}k')
    print(f'sum of post-horizon discounted cashflows (C_>h): $ {discountedHorizon}k')
    printmd(f'total discounted cashflows (C): **$ {summedDiscounted+discountedHorizon}k**')

In [6]:
interact(valuation,\
         timeHorizon = widgets.IntSlider(min=1,max=30,step=1,value=15,continuous_update=False),\
         periodsPerYear = widgets.SelectionSlider(options=[1,4, 12, 52],value=4),\
         discountRate = widgets.FloatSlider(min=0.1,max=25.1,step=0.1,value=5,continuous_update=False),\
         stakerFee = widgets.FloatSlider(min=0.1,max=0.9,step=0.05,value=0.5,continuous_update=False),\
         mid=widgets.FloatSlider(min=0,max=30,step=0.1,value=5,continuous_update=False),\
         slope=widgets.FloatSlider(min=0,max=3,step=0.1,value=1,continuous_update=False),\
         limit=widgets.IntText(value=12000000,continuous_update=False),\
         avgTransactionValue = widgets.IntSlider(min=1,max=30,step=1,value=5,continuous_update=False)
        )

interactive(children=(IntSlider(value=15, continuous_update=False, description='timeHorizon', max=30, min=1), …

<function __main__.valuation(timeHorizon, periodsPerYear, discountRate, stakerFee, mid, slope, limit, avgTransactionValue)>